In [ ]:
#######################################################
# Script:
#    testDecisionTree.py
# Usage:
#    python testDecisionTree.py
# Description:
#    Test the prediction model using test data set
# Authors:
#    Jackie Chu,   cchu@salesforce.com
#    Jasmin Nakic, jnakic@salesforce.com
#######################################################

import numpy as np
from sklearn import tree
from sklearn.externals import joblib

In [ ]:
# Enable or disable debug printing
debugFlag = False

# Feature list
perfCols = ["PageTime_ms","TotalServerTime_ms","TotalBrowserTime_ms","Action_count","Api_count","Db_count","DbTime_ms","Xhr_count"]

In [ ]:
# Add a set of columns to the destination
def addColumns(dest, src, colNames):
    # Initialize temporary array
    tmpArr = np.empty(src.shape[0])
    cols = 0
    # Copy column content
    for name in colNames:
        if cols == 0: # first column
            tmpArr = np.copy(src[name])
            tmpArr = np.reshape(tmpArr,(-1,1))
        else:
            tmpCol = np.copy(src[name])
            tmpCol = np.reshape(tmpCol,(-1,1))
            tmpArr = np.append(tmpArr,tmpCol,1)
        cols = cols + 1
    return np.append(dest,tmpArr,1)
#end addColumns

In [ ]:
# Get predictions using Decision Tree model
def getPredictions(data,colList,modelName):
    # Prepare the data for the model
    X = np.zeros(data.shape[0])
    X = np.reshape(X,(-1,1))
    X = addColumns(X,data,colList)
    if debugFlag:
        print("X 0: ", X[0:5])
    Y = np.copy(data["Status"])
    if debugFlag:
        print("Y 0: ", Y[0:5])

    modelFileName = modelName+".model"
    model = joblib.load(modelFileName)

    print("NAMES: ", data.dtype.names)
    print("TREE: ", model.tree_)
    print("MAX_DEPTH: ", model.tree_.max_depth)
    print("NODE_COUNT: ", model.tree_.node_count)
    print("CHILDREN_LEFT: ", model.tree_.children_left)
    print("CHILDREN_RIGHT: ", model.tree_.children_left)
    print("FEATURE: ", model.tree_.feature)
    print("THRESHOLD: ", model.tree_.threshold)
    print("FEATURE_IMPORTANCES: ", model.feature_importances_)
    print("APPLY: ", model.apply(X[:1]))
    print("DECISION PATH: ", model.decision_path(X[:1]))

    P = model.predict(X)
    print("SCORE values: ", model.score(X,Y))
    if debugFlag:
        print("P 0-5: ", P[0:5])

    return P
#end getPredictions

In [ ]:
# Write test results to a file
def writeResult(output,data,p):
    result = np.array(
       np.empty(data.shape[0]),
       dtype=[
           ("PageTime_ms",int),
           ("TotalServerTime_ms",int),
           ("TotalBrowserTime_ms",int),
           ("Action_count",int),
           ("Api_count",int),
           ("Db_count",int),
           ("DbTime_ms",int),
           ("Xhr_count",int),
           ("Status","|U20"),
           ("PREDICTION","|U20")
        ]
    )
    result["PageTime_ms"]     = data["PageTime_ms"]
    result["TotalServerTime_ms"]     = data["TotalServerTime_ms"]
    result["TotalBrowserTime_ms"]     = data["TotalBrowserTime_ms"]
    result["Action_count"]     = data["Action_count"]
    result["Api_count"]    = data["Api_count"]
    result["Db_count"]    = data["Db_count"]
    result["DbTime_ms"]    = data["DbTime_ms"]
    result["Xhr_count"]    = data["Xhr_count"]
    result["Status"] = data["Status"]
    result["PREDICTION"] = p
    if debugFlag:
        print("R 0-5: ", result[0:5])
    hdr = "PageTime_ms,TotalServerTime_ms,TotalBrowserTime_ms,Action_count,Api_count,Db_count,DbTime_ms,Xhr_count,Status,PREDICTION"
    np.savetxt(output,result,fmt="%s",delimiter=",",header=hdr,comments="")
#end writeResult


In [ ]:
# Start
inputFileName = "PerfRun_TestData.csv"
outputFileName = "PerfRun_TestResult.txt"
modelName = "PerfRun"

# All input columns - data types are strings, float and int
testData = np.genfromtxt(
    inputFileName,
    delimiter=',',
    names=True,
    dtype=("|U20",int,int,int,int,int,int,int,int)
)
if debugFlag:
    print("testData 0: ", testData[0:5])


In [10]:
# Get Prediction for PerfRun_TestData.csv
P = getPredictions(testData,perfCols,modelName)


X 0:  [[   0. 2198. 2041. 1709.   13.   52.  289.  502.   10.]
 [   0. 2204. 2023. 1679.   13.   50.  289.  525.   10.]
 [   0. 2723. 2182. 1944.   13.   50.  331.  472.   10.]
 [   0. 2402. 2178. 1716.   13.   52.  293.  511.   10.]
 [   0. 2323. 2199. 1675.   13.   54.  290.  491.   10.]]
Y 0:  ['Success' 'Success' 'Invalid' 'Success' 'Success']
NAMES:  ('Status', 'PageTime_ms', 'TotalServerTime_ms', 'TotalBrowserTime_ms', 'Action_count', 'Api_count', 'Db_count', 'DbTime_ms', 'Xhr_count')
TREE:  <sklearn.tree._tree.Tree object at 0x1a0b2e2308>
MAX_DEPTH:  3
NODE_COUNT:  15
CHILDREN_LEFT:  [ 1  2  3 -1 -1  6 -1 -1  9 10 -1 -1 13 -1 -1]
CHILDREN_RIGHT:  [ 1  2  3 -1 -1  6 -1 -1  9 10 -1 -1 13 -1 -1]
FEATURE:  [ 1  4  1 -2 -2  2 -2 -2  7  3 -2 -2  1 -2 -2]
THRESHOLD:  [ 2.4995e+03  1.2500e+01  2.0075e+03 -2.0000e+00 -2.0000e+00  2.6170e+03
 -2.0000e+00 -2.0000e+00  3.7750e+02  1.6930e+03 -2.0000e+00 -2.0000e+00
  2.7245e+03 -2.0000e+00 -2.0000e+00]
SCORE values:  0.6656346749226006
SCOR

In [11]:
# Write result to file PerfRun_TestResult.txt
writeResult(outputFileName,testData,P)

R 0-5:  [('52.0', 2198, 2041, 1709, 13, 52, 289, 502, 10, 'Success', 'Success')
 ('285.0', 2204, 2023, 1679, 13, 50, 289, 525, 10, 'Success', 'Success')
 ('5.5265081749e-313', 2723, 2182, 1944, 13, 50, 331, 472, 10, 'Invalid', 'Regression')
 ('0.0', 2402, 2178, 1716, 13, 52, 293, 511, 10, 'Success', 'Success')
 ('2.67e-322', 2323, 2199, 1675, 13, 54, 290, 491, 10, 'Success', 'Success')]
